This notebook fetches BDM Trial specs from the Google Sheet and then generates documentations in quarto markdown format.

The generated markdown files are saved in the `assets/yml/tables/` folder.

Install pandas, gspread, and PyYAML before running this notebook:

```bash
pip install pandas gspread PyYAML
```


In [ ]:
# Imports and setup

import os
from pathlib import Path
import pandas as pd
import yaml

In [ ]:
# Parameters

OUTPUT_DIR = Path('assets/auto-generated/')  # where to write the generated files
TRIAL_SPEC_SHEET_ID = os.environ.get('TRIAL_SPEC_SHEET_ID', '')

In [ ]:
# Helper functions

def camel_to_dash(camel_case_string):
  """Converts a CamelCase string to a dash-separated lower case string.

  Args:
    camel_case_string: The CamelCase string to convert.

  Returns:
    The dash-separated lower-case string.
  """
  import re

  if pd.isna(camel_case_string):
    raise ValueError('camel_case_string cannot be None')

  if camel_case_string.strip() == '':
    return camel_case_string.strip()

  # replace spaces with empty string, and prefix uppercase letters with a dash
  s = re.sub('(?!^)([A-Z]+)', r'-\1', camel_case_string.replace(' ', ''))
  return s.lower()

def get_sheet(sheet_id: str, sheet_name, backend: str='requests'):
    """Download a Google Sheets table as a pandas DataFrame."""
    url = ('https://docs.google.com/spreadsheets/d/'
           '{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}')
    url = url.format(sheet_id=sheet_id, sheet_name=sheet_name)
    if backend.lower() == 'requests':
      import requests
      from io import StringIO
      content = requests.get(url).content
      data = pd.read_csv(StringIO(content.decode('utf-8')))
    else:
      data = pd.read_csv(url)
    return data


def convert_sheet_to_yaml(table_info: pd.Series, root_dir) -> Path:
    """Write a pandas DataFrame to a CSV file."""

    assert table_info['name'] is not None, 'table name is required'

    file_name = camel_to_dash(table_info['name']) + '.yml'
    output_path = root_dir / file_name

    # append the category to the file name if it exists
    if (pd.notna(table_info['category']) and len(table_info['category']) > 0):
      category = camel_to_dash(table_info['category'])
      output_path = root_dir / category / file_name

    output_path.parent.mkdir(parents=True, exist_ok=True)

    df = get_sheet(TRIAL_SPEC_SHEET_ID, table_info['name'])

    with open(output_path, 'w') as f:

      # FIXME - this is a hack to handle the fact that the glossary description is in the same sheet as the glossary
      glossary_categories_df = None
      if 'glossary' in df['data_type'].unique():
        glossary_categories_df = df[df['data_type'] == 'glossary']
        df = df.drop(glossary_categories_df.index)

      data = df.groupby('category', sort=False).apply(
        lambda x: x.to_dict(orient='records'), include_groups=False).to_dict()

      # reformat data to be a list of dictionaries
      data = [{'category': c, 'fields': v} for  c,v in data.items()]

      # FIXME - this is a hack to handle the fact that the glossary description is in the same sheet as the glossary - see above too
      if glossary_categories_df is not None:
        for d in data:
          catg = d['category']
          if catg in glossary_categories_df['name'].values:
            catg_info = glossary_categories_df.query('name == @catg').iloc[0].to_dict()
            # TODO append all columns
            d['description'] = catg_info['description'] if pd.notna(catg_info['description']) else ''
            d['publish'] = catg_info['publish'] if pd.notna(catg_info['publish']) else False

      yaml.dump(
        data,
        f, default_flow_style=False, indent=2, sort_keys=False)

    return output_path

In [ ]:
# Run the pipeline

tables = get_sheet(TRIAL_SPEC_SHEET_ID, 'Tables')
tables = tables.query('publish == True')
tables['category'] = tables['category'].str.split('; ')
tables = tables.explode('category').reset_index(drop=True)

tables.apply(convert_sheet_to_yaml, axis=1, root_dir=OUTPUT_DIR) # type: ignore